In [1]:
import os
import math
import optuna
import pathlib
import pickle
import mlflow
import pathlib
import pandas as pd
import xgboost as xgb
from dotenv import load_dotenv
from optuna.samplers import TPESampler
from mlflow.models.signature import infer_signature
from sklearn.metrics import root_mean_squared_error
from sklearn.feature_extraction import DictVectorizer
from prefect import flow, task

@task(name="Read Data")
def read_data(file_path: str) -> pd.DataFrame:
    """Read data into DataFrame"""
    df = pd.read_parquet(file_path)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df

@task(name="Add Features")
def add_features(df_train: pd.DataFrame, df_val: pd.DataFrame):
    """Add features to the model"""
    df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
    df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

    categorical = ["PU_DO"]  #'PULocationID', 'DOLocationID']
    numerical = ["trip_distance"]

    dv = DictVectorizer()

    train_dicts = df_train[categorical + numerical].to_dict(orient="records")
    X_train = dv.fit_transform(train_dicts)

    val_dicts = df_val[categorical + numerical].to_dict(orient="records")
    X_val = dv.transform(val_dicts)

    y_train = df_train["duration"].values
    y_val = df_val["duration"].values
    return X_train, X_val, y_train, y_val, dv

@task(name="Hyperparameter Tunning")
def hyper_parameter_tunning(X_train, X_val, y_train, y_val, dv):
    
    mlflow.xgboost.autolog()
    
    training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
    
    validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")
    
    train = xgb.DMatrix(X_train, label=y_train)
    
    valid = xgb.DMatrix(X_val, label=y_val)
    
    # ------------------------------------------------------------
    # Definir la función objetivo para Optuna
    #    - Recibe un `trial`, que se usa para proponer hiperparámetros.
    #    - Entrena un modelo con esos hiperparámetros.
    #    - Calcula la métrica de validación (RMSE) y la retorna (Optuna la minimizará).
    #    - Abrimos un run anidado de MLflow para registrar cada trial.
    # ------------------------------------------------------------
    def objective(trial: optuna.trial.Trial):
        # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
        # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
        params = {
            "max_depth": trial.suggest_int("max_depth", 4, 100),
            "learning_rate": trial.suggest_float("learning_rate", math.exp(-3), 1.0, log=True),
            "reg_alpha": trial.suggest_float("reg_alpha",   math.exp(-5), math.exp(-1), log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", math.exp(-6), math.exp(-1), log=True),
            "min_child_weight": trial.suggest_float("min_child_weight", math.exp(-1), math.exp(3), log=True),
            "objective": "reg:squarederror",  
            "seed": 42,                      
        }

        # Run anidado para dejar rastro de cada trial en MLflow
        with mlflow.start_run(nested=True):
            mlflow.set_tag("model_family", "xgboost")  # etiqueta informativa
            mlflow.log_params(params)                  # registra hiperparámetros del trial

            # Entrenamiento con early stopping en el conjunto de validación
            booster = xgb.train(
                params=params,
                dtrain=train,
                num_boost_round=100,
                evals=[(valid, "validation")],
                early_stopping_rounds=10,
            )

            # Predicción y métrica en validación
            y_pred = booster.predict(valid)
            rmse = root_mean_squared_error(y_val, y_pred)

            # Registrar la métrica principal
            mlflow.log_metric("rmse", rmse)

            # La "signature" describe la estructura esperada de entrada y salida del modelo:
            # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
            # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
            signature = infer_signature(X_val, y_pred)

            # Guardar el modelo del trial como artefacto en MLflow.
            mlflow.xgboost.log_model(
                booster,
                name="model",
                input_example=X_val[:5],
                signature=signature
            )

        # Optuna minimiza el valor retornado
        return rmse

    # ------------------------------------------------------------
    # Crear el estudio de Optuna
    #    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
    #    - direction="minimize" porque queremos minimizar el RMSE.
    # ------------------------------------------------------------
    sampler = TPESampler(seed=42)
    study = optuna.create_study(direction="minimize", sampler=sampler)

    # ------------------------------------------------------------
    # Ejecutar la optimización (n_trials = número de intentos)
    #    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
    #    - Abrimos un run "padre" para agrupar toda la búsqueda.
    # ------------------------------------------------------------
    with mlflow.start_run(run_name="XGBoost Hyperparameter Optimization (Optuna)", nested=True):
        study.optimize(objective, n_trials=3)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    # Asegurar tipos/campos fijos (por claridad y consistencia)
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"

    return best_params

@task(name="Train Best Model")
def train_best_model(X_train, X_val, y_train, y_val, dv, best_params) -> None:
    """train a model with best hyperparams and write everything out"""

    with mlflow.start_run(run_name="Best model ever"):
        train = xgb.DMatrix(X_train, label=y_train)
        valid = xgb.DMatrix(X_val, label=y_val)

        mlflow.log_params(best_params)

        # Etiquetas del run "padre" (metadatos del experimento)
        mlflow.set_tags({
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "optuna",
            "model_family": "xgboost",
            "feature_set_version": 1,
        })

        # --------------------------------------------------------
        # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
        #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
        # --------------------------------------------------------
        booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=10,
        )

        # Evaluar y registrar la métrica final en validación
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

        # --------------------------------------------------------
        # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
        # --------------------------------------------------------
        pathlib.Path("preprocessor").mkdir(exist_ok=True)
        with open("preprocessor/preprocessor.b", "wb") as f_out:
            pickle.dump(dv, f_out)

        mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
        feature_names = dv.get_feature_names_out()
        input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names)

        # Para que las longitudes coincidan, usa el mismo slice en y_pred
        signature = infer_signature(input_example, y_val[:5])

        # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.xgboost.log_model(
            booster,
            name="model",
            input_example=input_example,
            signature=signature,
        )
    return None

@flow(name="Main Flow")
def main_flow(year: int, month_train: str, month_val: str) -> None:
    """The main training pipeline"""
    
    train_path = f"../data/green_tripdata_{year}-{month_train}.parquet"
    val_path = f"../data/green_tripdata_{year}-{month_val}.parquet"
    
    load_dotenv(override=True)  # Carga las variables del archivo .env
    EXPERIMENT_NAME = "/Users/ivan.morales@iteso.mx/nyc-taxi-experiment-prefect"

    mlflow.set_tracking_uri("databricks")
    experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

    # Load
    df_train = read_data(train_path)
    df_val = read_data(val_path)

    # Transform
    X_train, X_val, y_train, y_val, dv = add_features(df_train, df_val)
    
    # Hyper-parameter Tunning
    best_params = hyper_parameter_tunning(X_train, X_val, y_train, y_val, dv)
    
    # Train
    train_best_model(X_train, X_val, y_train, y_val, dv, best_params)

if __name__ == "__main__":
    main_flow(year=2025, month_train="01", month_val="02")

21:15:56.931 | INFO    | prefect - Starting temporary server on http://127.0.0.1:8145
See https://docs.prefect.io/v3/concepts/server#how-to-guides for more information on running a dedicated Prefect server.

21:16:03.849 | INFO    | Flow run 'obedient-jaybird' - Beginning flow run 'obedient-jaybird' for flow 'Main Flow'

2025/10/28 21:16:07 INFO mlflow.tracking.fluent: Experiment with name '/Users/ivan.morales@iteso.mx/nyc-taxi-experiment-prefect' does not exist. Creating a new experiment.


21:16:08.551 | INFO    | Task run 'Read Data-2b1' - Finished in state Completed()

21:16:08.990 | INFO    | Task run 'Read Data-ac5' - Finished in state Completed()

...

2025/10/28 21:18:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/28 21:18:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\ivanm\5to Semestre\Proyecto de Ciencia de Datos\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:169: UserWarning: [21:18:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats."
2025/10/28 21:18:19 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
c:\Users\ivanm\5to Semestre\Proyecto de Ciencia de Datos\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:169: UserWarning: [21:18:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in th

c:\Users\ivanm\5to Semestre\Proyecto de Ciencia de Datos\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:18:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


🏃 View run Best model ever at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2765897724609619/runs/620695c289764c8998494dd654e38a89
🧪 View experiment at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2765897724609619


21:18:59.065 | INFO    | Task run 'Train Best Model-298' - Finished in state Completed()

21:18:59.111 | INFO    | Flow run 'obedient-jaybird' - Finished in state Completed()